# One-hot encoding demonstration
expected memory usage: 2.3GB.  
expected runtime: 15 seconds.

##  Introduction
This notebook demonstrates calculation of one-hot encoding under homomorphic encryption. Given a ciphertext and a list of the possible values that it can contain, we calculate a list of indicator ciphertexts that indicates which slots in the source ciphertext contain each value. Formally, the i'th resulted indicator ciphertext will contains 1 in slot j only if the j'th slot in the source ciphertext contains the i'th value, and 0 otherwise.

### 1. We start with some imports:

In [ ]:
import numpy as np
import pyhelayers
import utils 

utils.verify_memory()

NUM_ELEMENTS_TO_SHOW = 16

### 2. Now we initialize the context

In [ ]:
requirement = pyhelayers.HeConfigRequirement(
    num_slots = 2**14,
    multiplication_depth = 10,
    fractional_part_precision = 50,
    integer_part_precision = 10)

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)
print(he_context.print_signature())

### 3. Create input for the demonstration and calculate the expected output in plain

In [ ]:
enc = pyhelayers.Encoder(he_context)

possible_values = [0,1,2]
input = np.random.choice(possible_values, size=he_context.slot_count())

expected = [np.array([1 if elem==val else 0 for elem in input]) for val in possible_values]

src = enc.encode_encrypt(input)

print("Showing first ", NUM_ELEMENTS_TO_SHOW, " elements...\n")
print("Input for the demonstration:\t", input[:NUM_ELEMENTS_TO_SHOW], "\n")
for i in range(len(possible_values)):
    print("Expected one-hot encoding for value ", possible_values[i],": ", expected[i][:NUM_ELEMENTS_TO_SHOW])

### 4. Calculate one-hot encoding under homomorphic encryption

In [ ]:
fe = pyhelayers.FunctionEvaluator(he_context)

with utils.elapsed_timer('one-hot', 1) as timer:
    res = fe.one_hot(src, possible_values)

### Results

In [ ]:
print("Showing first ", NUM_ELEMENTS_TO_SHOW, " elements...\n")
print("Input for the demonstration:\t", input[:NUM_ELEMENTS_TO_SHOW], "\n")

for i in range(len(possible_values)):
    plain_res = enc.decrypt_decode_int(res[i])
    print("One-hot encoding calculated under FHE for value ", possible_values[i],":\t", plain_res[:NUM_ELEMENTS_TO_SHOW])
    print("Expected one-hot encoding for value ", possible_values[i],":\t\t", expected[i][:NUM_ELEMENTS_TO_SHOW], "\n")
    np.testing.assert_array_equal(plain_res, expected[i])

In [ ]:
print("RAM usage:", utils.get_used_ram(), "MB")